In [ ]:
#| echo: false
#| output: false

using Pkg

Pkg.activate("./")

Pkg.add([
	"Random",
	"StatsBase",
	"StatsPlots",
	"LaTeXStrings",
	"ParetoSmooth",
	"ForwardDiff",
	"Distributions"
])

using Random, Distributions, StatsPlots, LaTeXStrings, Dates

Random.seed!(123);

START_TIME = now() # Used for timing the whole execution.

::: {.callout-tip}
## Post insipration

This post was heavily inspired by the work by @psis on Pareto Smoothed Importance Sampling, which tackles the related problem of how to deal with extreme values in importance sampling.
:::

## A day in the life of a data scientist

Picture the situation: you're a data scientist / statistician / economist at an e-commerce company. You're working with a team who plan to run an A/B test to evaluate the impact of a new feature or promotion, and critically they want to evaluate the impact of the change on **revenue**. Sounds easy, so you get to work! 

Your randomisation and analysis units are visitors, and since the goal is to measure the impact on total revenue, your main metric is revenue per visitor. You plan to perform a t-test against the null hypothesis that the average revenue per visitor is the same in both control and treatment, and if you reject the null hypothesis you'll report the effect size, with a confidence interval, and pop the champaign!

As a responsible experimenter, you kick things off with a power analysis, to estimate the required sample size you'll need for this test. To do this you run a query to get the variance of the revenue per visitor over the last few weeks. And here, things start to go wrong... your variance is huge! At this rate, you'll have to run your test for months in order to measure any reasonable effect. 

So now what? You plot a histogram of your revenue per visitor metric to see what's going on, and you see something like this:


In [ ]:
#| label: fig-revenue-samples
#| fig-cap: 'Hypothetical revenue per visitor histogram. Skewed, with lots of zeros on one side and a few massive outliers on the other side.'

data_dist = MixtureModel(
	[
	   LogNormal(2.0, 1.5),
	   GeneralizedPareto(0.0, 40.0, 0.7)
	], 
	[0.9, 0.1]
);

x_samples = rand(data_dist, 1000);

p1 = histogram(
     x_samples, 
     normalize=false,
	 legend=false
)
xlabel!("Revenue per Visitor")
plot(p1, size=(400,300))

And here's the problem. The distribution of the metric is highly skewed, with lots of zeros, then some "normal" spenders, and then a very small number of extreme spenders - we'll call these our outliers. And it's these outliers that are disproportionately increasing the variance of your average revenue per visitor estimate, hurting your power, and eventually making you wait months to estimate the impact of your experiment. 

## Outliers in A/B testing

The previous section was a hypothetical story, but I think it's quite a common scenario. But what can be done? My usual steps are:

1. First, check if the outliers are real. It's often the case that outliers are actually just bad data, in which it might be reasonable to exclude them or fix them.
1. Do some variance reduction! [Cuped](https://booking.ai/how-booking-com-increases-the-power-of-online-experiments-with-cuped-995d186fff1d) is a very effective method of reducing the variance of your metric based on data from before the start of your experiment. It's always worth trying and it's about as close to a free lunch as you can get when it comes to improving A/B testing power. However, cuped reduces variance, not skew, so it's likely your power will still be hindered by the outliers.
1. If the previous two steps doesn't reduce your variance sufficiently, then it's time to look at trimming or winsorizing. These are very powerful techniques, but come with a large drawback: **you are introducing bias into your estimates**. This is especially problematic because here we are trimming or winsorizing on one side only (removing extremely high values) - you have no outliers on the lower side of the distribution.

::: {.callout-warning collapse="true"}
## Don't log transform!!

On almost every blog I see, when the issue of a skewed metric arises, someone will recommend log-transforming the metric. While this will certainly make your metric less skewed, and may improve power, it's a big no for financial metrics like revenue. 

This is because when you transform your metric, you also change the null hypothesis you are testing and therefore the interpretation of any effect you measure. For revenue, you really care about the effect on the **(arithmetic) mean** revenue per visitor, because improving this is what will translate into improving total revenue. However, the log transform means you are testing for an effect on the **geometric mean**, which is not the same and improving it does not necessarily translate into improving your bottom line, leading to potentially bad business decisions.
:::

Trimming and winsorizing are specifically designed to deal with outliers and make your estimator more robust. With trimming, you filter out all samples that are above some threshold, while with winsorizing you apply a cap, so that all values above a threshold are capped to that threshold. Both are very effective at reducing  variance, however because you are removing or capping the top X data points, you introduce a potentially large negative bias, so I usually treat it as a bit of a last resort.

But is this the best we can do? In this post I'm going to explore how we might formalise this problem with extreme value theory, and see if we can find some estimators that perform better than the trimmed or winsorized means. Here, by "better", I mean achieving similar levels of variance reduction, while introducing less bias.

## Extreme value theory and power laws

::: {.callout-warning}
## Disclaimer 

I'm far from an expert in extreme value theory, I've really only learned about the topic in the last few weeks. Take this section with a pinch of salt. 
:::

*Extreme value theory is a branch of statistics dealing with the extreme deviations from the median of probability distributions.*^[Sorry, this definition is fully plagiarised from [Wikipedia](https://en.wikipedia.org/wiki/Extreme_value_theory#:~:text=Extreme%20value%20theory%20or%20extreme,extreme%20than%20any%20previously%20observed.), but it sounds relevant!] This is exactly what we are dealing with here - outliers are messing with our statistics. So perhaps there are some tools from extreme value theory that we can use to reduce the bias in our trimmed / winsorized mean? Say hello to the second theorem in extreme value theory:

> **Second Theorem in Extreme Value Theory**
> 
> For a wide range of univariate probability distributions, the tail of that distribution can be well modelled by the Generalised Pareto distribution (GPD).
> 
> -- *@pickands1975statistical, @balkema1974residual*

This sounds relevant! Perhaps we can describe the outliers in our skewed revenue  distribution as a GPD. So what does a GPD look like? 

### The Pareto and Generalised Pareto distributions

The GPD is a more flexible version of the well known Pareto distribution [@powerlaws], also known as the 80-20 rule. Both the GPD and Pareto distribution are characterised by having "power law" tails, where for large X the distribution tends towards:

$$
P(x) \rightarrow x^{-m} \quad \text{as} \quad x \rightarrow \infty
$$

::: {.callout-tip collapse="true"}
## The maths

The probability density function of the Pareto distribution is given by:
$$
P_\text{pareto}(x | \alpha, \theta) = \frac{\alpha \theta^\alpha}{x^{\alpha-1}}
$$
This distribution has two parameters, $\alpha$ and $\theta$:

* $\theta$ controls the minimum of the distribution: $x \in [\theta, \infty)$
* $\alpha$ controls the slope of the distribution.

The probability density function of the GPD is given by:
$$
P_\text{GPD}(x | \mu, \sigma, \xi) = \frac{1}{\sigma} \left( 1 + \xi \frac{x - \mu}{\sigma} \right)^{-(1/\xi + 1)}
$$

The parameters of this distribution are more complex. In this post I'm going to only look at cases where $\xi > 0$. In this case:

* $\mu$ controls the minimum of the distribution: $x \in [\mu, \infty)$
* $\sigma$ controls the scale of the distribution. 
* $\xi$ controls the slope of the power law tail of the distribution. Comparing it to $\alpha$ for the Pareto distribution, $\xi = 1 / \alpha$.

In the case where $\mu = \sigma / \xi$ the GPD is the same as the Pareto distribution.
:::

You can see the power law tails in @fig-pareto below, where I compare the Pareto and GPD distribution with the log-normal distribution, which does not have a power law tail:


In [ ]:
#| label: fig-pareto
#| fig-cap: 'A comparison between the pareto distribution (red), the Generalised Pareto distribution (GPD) (blue), and the log-normal distribution (green). Both axes are log scaled. Pareto has parameters alpha=1.5 and theta=20. GPD has parameters mu=100.0, theta=20.0 and xi=2/3. Log-normal has parameters mu=4.5, sigma=1.0.'

gpd = GeneralizedPareto(100.0, 20.0, 1.0 / 1.5)
pd = Pareto(1.5, 20.0)
log_norm = LogNormal(4.5, 1.0)

plot(gpd, label="GPD", size=(400,200))
plot!(pd, label="Pareto")
plot!(log_norm, label="Log-Normal")
plot!(xscale=:log10, yscale=:log10, ylabel="P(x)", xlabel="x", xrange=[1.0; 10000.0], yrange=[0.0000001; 0.4], legend=:bottomleft)

The distributions are plotted on a log-log scale, and you can see that both the Pareto and GPD distributions are linear for large values of $x$ (indicating a power law), while the log-normal distribution never becomes linear, it is always curving downwards.

The GPD and Pareto distribution differ at small values of $x$, where the GPD is more flexible and can curve up or down, while the Pareto distribution remains a pure power law for all values of $x$.

### Infinite Variance

Power law type distributions are interesting because depending on the slope of the tail, the population might have infinite mean or infinite variance. Specifically for the GPD and Pareto distributions we have the following three cases:

|                                   | Pareto              | GPD                |
|-----------------------------------|---------------------|--------------------|
| 1. Infinite Mean and Variance     | $\alpha \leq 1$     | $\xi \geq 1$       |
| 2. Finite Mean, Infinite Variance | $1 < \alpha \leq 2$ | $0.5 < \xi \leq 1$ |
| 3. Finite Mean and Variance       | $\alpha > 2$        | $\xi \leq 0.5$     |

: Three important scenarios for the sample mean and variance of the GPD and Pareto distributions. {#tbl-pareto-cases}

**But what does infinite mean or infinite variance really mean?** The *sample* mean or variance is always finite, even if the population distribution from which that sample was drawn has infinite mean or variance. However, if the population distribution has infinite mean / variance, then the sample mean / variance will *not* converge as the sample size approaches infinity. 

This can be seen in @fig-inf-variance below, where I simulate estimating the sample variance as a function of sample size for the Log-Normal and GPD distributions, where the GPD distribution has infinite variance. Each grey line represents one sample, where I calculate the sample variance cumulatively, adding one point at a time. 


In [ ]:
#| label: fig-inf-variance
#| fig-cap: 'The sample variance vs sample size for the GPD (left) and log-normal distribution (right). The GPD used here has infinite population variance, so the sample variance does not converge as sample size increases.'

function cumvar(x::Vector{Float64})
	sum_x = cumsum(x)
	sum_x2 = cumsum(x.^2)
	n = 1:length(x)

	var = sum_x2 ./ n - (sum_x ./ n).^2
	return var
end

function plot_var_vs_samplesize(dist::UnivariateDistribution, max_samples::Int64, title::AbstractString)
	p = plot(
		xscale = :log10, 
		yscale=:log10, 
		xlabel="Sample Size", 
		ylabel="Sample Variance", 
		yrange=[0.5, 1e8], 
		title=title,
		titlefontsize=10
	)
	samples = [cumvar(rand(dist, max_samples)) for i in 1:20]

	plot!(
		1:max_samples, 
		samples[1], 
		color=:black, 
		alpha=0.2, 
		label="Samples"
	)
	plot!(
		1:max_sample_size, 
		samples[2:length(samples)], 
		color=:black, 
		alpha=0.2, 
		label=""
	)
	return p
end

max_sample_size = 100000

p1 = plot_var_vs_samplesize(gpd, max_sample_size, L"GPD(\mu=100, \theta=20.0, \xi=2/3)")
p2 = plot_var_vs_samplesize(log_norm, max_sample_size, L"LogNormal(\mu=4.5, \sigma=1.0)")
hline!([var(log_norm)], color=:red, label="Population Variance")

plot(p1, p2, fmt = :png; dpi = 600)

You can see that for the log-normal distribution, as the sample size gets large the variance converges towards the population variance, while for the GPD distribution it does not converge, and instead just keeps increasing. 

This is a problem, because when performing inference about the mean via the t-test, we construct the t-statistic from the sample mean and sample variance. If, for example, we are performing a one-sample t-test to test the null hypothesis that the mean of the population distribution is equal to a specific value $\mu_0$, we would calculate the t-statistic as:

$$
t = \frac{\hat{\mu} - \mu_0}{\hat{\sigma} / \sqrt{n}}
$$

Where $\hat{\mu}$ is the sample mean, $\hat{\sigma}^2$ is the sample variance and $n$ is the sample size. Therefore, referring back to the three cases from @tbl-pareto-cases:

* If we are in case (1) where both the mean and variance are infinite, then there is basically no hope for any inference about the mean, since it's infinite! If you have a distribution with such large outliers that you are in this case, then you'll always struggle to perform any inference about it's mean. I'm not even sure what the interpretation of such an analysis would be...
* If we are in case (3) then both the mean and variance are finite. This is trivial, and it's likely that the regular t-test based on the sample mean and variance will perform fine. Probably your data wouldn't even look like it has significant outliers.
* Case (2) is the interesting one. The mean is finite, so we may want to perform inference on it, however the variance is infinite, which will break the t statistic which uses the sample variance. 

So to summarise:

1. It's common for the tails of a wide variety of distributions to be well described by the GPD distribution.
1. Under certain conditions, the GPD distribution can have infinite variance but a finite mean. 
1. This would make any sample drawn from that distribution have a sample variance that does not converge as the sample size increases, and instead the variance will just increase as the sample size increases.
1. This will break out t-statistic, which requires the sample variance.

This sounds a lot like the situation we have with outliers in our revenue per visitor metric! Perhaps the tail of that distribution follows a power law, where we have infinite variance. This would explain why our variance is so large, and therefore why our experimental power is so poor! 

## Inference about the mean of a power law distribution

So if we cannot use the sample variance, what can we do? Well really we only use the sample variance so that we can estimated the standard error of the mean, $\hat{\sigma}_\mu$, as:

$$
\hat{\sigma}_\mu = \frac{\hat{\sigma}}{\sqrt{n}}
$$

Is there a way we can do this without the sample variance? Yes, with maximum likelihood estimation and the delta method! This approach takes three steps:

1. Assume a parametric distribution for your population
1. Fit your distribution to your data using maximum likelihood estimation
1. Calculate the mean and the standard error of the mean from your maximum likelihood fit. This requires calculating the hessian of the log-likelihood of your data, and using the delta method.

::: {.callout-tip collapse="true"}
## Detailed method with maths

1. You have a sample of size $n$: $\left[ X_1, X_2, X_3, ... X_n \right]$
1. Assume a parametric distribution for your population: $P(x|\theta)$, where $\theta$ is a vector of the parameters of the distribution.
1. Define the Log Likelihood as: $L(\theta) = \sum_{j=1}^n \ln{(P(X_j|\theta))}$
1. Estimate $\hat{\theta}$ by maximising $L(\theta)$: $\hat{\theta} = \text{arg max}_{\theta \in \Theta} L(\theta)$
1. Calculate the [hessian matrix](https://en.wikipedia.org/wiki/Hessian_matrix) of $L(\theta)$, $H(\theta)$. This is the matrix of second derivatives of $L$.
1. Use the hessian matrix to estimate the covariance matrix for the standard errors of $\hat{\theta}$ as 
$\hat{\Sigma} = -H(\hat{\theta})^{-1}$, from [@covmle].
1. Estimate the maximum likelihood estimate of the mean from $\hat{\theta}$, based on the parametric form for the mean of your chosen distribution $P(x|\theta)$: $\hat{\mu} = E(\hat{\theta})$.
1. Use the [delta method](https://en.wikipedia.org/wiki/Delta_method) to estimate the standard error on the mean, $\hat{\sigma}_\mu$, where $\Delta E(\theta)$ is the first derivative (jacobian) of the mean function:

$$
\hat{\sigma}_\mu^2 = \Delta E(\hat{\theta})^T \cdot \hat{\Sigma} \cdot \Delta E(\hat{\theta})
$$ {#eq-delta-multivariate}

Note: if the parameters are all independent, meaning the covariance matrix is a diagonal matrix, then the above equation simplifies to:

$$
\hat{\sigma}_\mu^2 = \sum_i \left| \frac{\partial E}{\partial \theta_i} \right|^2 \hat{\sigma}_i^2
$$ {#eq-delta-indep}

Where $\sigma_i^2$ are the diagonal elements of $\hat{\Sigma}$.
:::

This method is nice because it doesn't require us to estimate the sample variance at any point, so we are able to estimate the mean and standard error on the mean, even when the population variance is infinite!

This method does, however, have some drawbacks - you need to make an assumption about the parametric form of the population distribution, and if that is incorrect it's very likely the estimator will be biased. Also, even if the parametric form is correctly specified, it's usually the case that the maximum likelihood estimator is not unbiased by default, it usually requires some bias correction to achieve that. This goes beyond the scope of this blog post, but would certainly be possible to explore.

Now let's see how this will work with the GPD and Pareto distributions.

### The Pareto (Hill) estimator

While the more basic Pareto distribution might not have as much flexibility as the GPD, and therefore might not fit the tail of your distribution as well, the maximum likelihood estimate can be calculated analytically, making it much simpler and faster to implement. 

The maximum likelihood estimator of the Pareto distribution is often known as the Hill estimator, and the parameters are calculated as follows (see @paretoest for the derivation):

$$
\hat{\theta} = \min{(X_j)}
$$
$$
\hat{\alpha} = \frac{n}{\sum_{j=1}^n \ln{\frac{X_j}{\hat{\theta}}}} = \frac{n}{\left( \sum_{j=1}^n \ln{X_j} \right) - n \ln{\hat{\theta}}}
$$

The maximum likelihood estimator for the mean and standard error are then calculated from these parameters as:

::: {.callout-tip collapse="true"}
## Derivation of the mean and standard error

The mean of the Pareto distribution is given by:
$$
\mu(\theta, \alpha) = \frac{\theta \alpha}{\alpha - 1}
$$

The parameters are independent, so there is no covariance. Their standard errors are given by @malik1970estimation (via [wikipedia](https://en.wikipedia.org/wiki/Pareto_distribution#Estimation_of_parameters)) as:

$$
\hat{\sigma}_\alpha = \frac{\hat{\alpha}}{\sqrt{n}}
$$

$$
\hat{\sigma}_\theta = \frac{\theta \sqrt{\alpha n}}{(\alpha n - 1) \sqrt{\alpha n - 2}} \underset{n \rightarrow \infty}{\approx} \frac{\theta}{\alpha n}
$$

We can calculate the standard error on the mean using @eq-delta-indep:

$$
\hat{\sigma}_\mu^2 = \left| \frac{\partial \mu}{\partial \alpha} \right|^2 \hat{\sigma}_\alpha^2 + \left| \frac{\partial \mu}{\partial \theta} \right|^2 \hat{\sigma}_\theta^2
$$

The two derivatives are:

$$
\frac{\partial \mu}{\partial \alpha} = \frac{-\theta}{(\alpha - 1)^2}
$$

$$
\frac{\partial \mu}{\partial \theta} = \frac{\alpha}{(\alpha - 1)}
$$

So finally substituting it all in we can derive the following equation for the standard error on the mean:

$$
\hat{\sigma}_\mu = | \hat{\mu} | \sqrt{ \frac{1}{n(\alpha - 1)^2} +  \frac{1}{\alpha^2 n^2} }
$$
:::

$$
\hat{\mu}_\text{pareto} = \frac{\hat{\theta} \hat{\alpha}}{\hat{\alpha} - 1}
$$
$$
\hat{\sigma}_{\mu, \text{pareto}} = | \hat{\mu} | \sqrt{ \frac{1}{n(\alpha - 1)^2} +  \frac{1}{\alpha^2 n^2} }
$$

This is implemented in Julia below:


In [ ]:
#| output: false
struct EstimatorResult
	μ::Float64
	σ::Float64
end

function sample_mean_variance_estimator(x::Vector{Float64})
	μ = mean(x)
	n = length(x)
	σ = std(x) / sqrt(n)

	return EstimatorResult(μ, σ)
end

In [ ]:
#| echo: true
#| output: false
function hill_estimator(x::Vector{Float64})
	n = length(x)

	# Maximum likelihood estimates of the pareto parameters
	θ = minimum(x)
	α = n / (sum(log.(x)) - n * log(θ))

	if α <= 1.0
		# If alpha is less than 1, the mean is infinite. 
		# In this case this estimator won't work, so fall
		# back to the sample mean and variance.
		return sample_mean_variance_estimator(x)
	end

	# Estimate the mean and standard error from the MLE parameters
	μ = α * θ / (α - 1)
	σ = abs(μ) * sqrt( 1 / (n * (α - 1)^2) +  1 / (α * n)^2 )

	# Return a struct holding the estimated mean and standard error
	return EstimatorResult(μ, σ) 
end

### The GPD estimator

For the GPD estimator it's more complicated, because there is no closed form analytical solution to the maximum likelihood estimation. This is a well known challenge in extreme value theory and many methods have been proposed to fit the GPD to a sample. 

Here I will be using the method proposed by @zhang2009new, which was already implemented in the `ParetoSmooth.jl` package. I will then use the great automatic differentiation built into Julia in `ForwardDiff.jl` to estimate the gradient and hessian, which I use to estimate the standard error on the mean via @eq-delta-multivariate. 

The full implementation in Julia is here:


In [ ]:
#| echo: true
#| output: false
using ParetoSmooth: gpd_fit
using ForwardDiff: gradient, hessian

function gpd_log_pdf(params::AbstractVector, x::Vector{Float64})
	sum(logpdf.(GeneralizedPareto(params...), x))
end

function gpd_mean(params::AbstractVector)
	mean(GeneralizedPareto(params...))
end

function gpd_estimator(x::Vector{Float64})
	n = length(x)

	# Estimate the GPD parameters numerically
	u = minimum(x)
	x_centered = x .- u
	ξ, σ = gpd_fit(x_centered, 1.0; sort_sample=true, wip=false)
	params = [u, σ, ξ]

	if ξ >= 1.0
		# If xi is greater than 1, the mean is infinite. 
		# In this case this estimator won't work, so fall
		# back to the sample mean and variance.
		return sample_mean_variance_estimator(x)
	end

	# Perform the automatic differentiation to get the 
	# required gradient and hessian
	H = hessian(z -> gpd_log_pdf(z, x), params)
	Σ = (-1 * H) ^ -1
	dμ = gradient(gpd_mean, params)

	# Estimate the mean and the standard error
	μ = gpd_mean(params)
	v = transpose(dμ) * Σ * dμ

	# Return a struct holding the estimated mean and standard error
	return EstimatorResult(μ, sqrt(v))
end

As an aside, this really shows off some of the strengths of Julia - I was able to implement this quite complex method in such a short and easy to read piece of code. However, as it's a numerical solution it will run slower and be less stable than the analytical Hill estimator for the Pareto distribution.

### How do these estimators perform?

To evaluate these estimators, I ran some simulations. First I simulated data coming from the Pareto and GPD distributions, with finite mean but infinite variance. Since I simulate this data I know the true population mean, so I can compare the estimators and see how well they do at estimating the population mean. I then repeat this process many times, each time collecting the estimated population mean and standard error. 

I compare three estimators:

1. **The sample mean / variance estimator.** This is the standard method of estimating the population mean used in the t-test. 
1. **The Hill estimator.** This is the maximum likelihood estimator based on the assumption that the data comes from a Pareto distribution.
1. **The GPD estimator.** This is the maximum likelihood estimator based on the assumption that the data comes from a GPD distribution. 


In [ ]:
const EstimatorResults = Vector{EstimatorResult}
μ(results::EstimatorResults) = map(r->r.μ, results)
σ(results::EstimatorResults) = map(r->r.σ, results)

function ci(r::EstimatorResult, α::Float64, n_samples::Int64)
	d = TDist(n_samples - 1)
	t_crit = quantile(d, 1.0 - α/2.0)
	lb = r.μ - t_crit * r.σ
	ub = r.μ + t_crit * r.σ
	return lb, ub
end

struct Simulation
	ests::EstimatorResults
	μ_true::Float64
	n_samples::Int64
	function Simulation(
		pop_dist::UnivariateDistribution, 
		n_samples::Int64,
		n_iterations::Int64,
		estimator,
		estimator_args=()
		)

		μ_true = mean(pop_dist)
		estimates = [
			estimator(rand(pop_dist, n_samples), estimator_args...)
			for i in 1:n_iterations
		]

		return new(estimates, μ_true, n_samples)
	end
end

function standardized_error(sim::Simulation)
	(μ(sim.ests) .- sim.μ_true) ./ σ(sim.ests)
end

function mse(sim::Simulation)
	mean((μ(sim.ests) .- sim.μ_true).^2)
end

function variance(sim::Simulation)
	std(μ(sim.ests))^2
end

function bias(sim::Simulation)
	mean(μ(sim.ests) .- sim.μ_true)^2
end

function coverage(sim::Simulation, α::Float64)
	mean([
		begin
			lbub = ci(r, α, sim.n_samples)
			(sim.μ_true >= lbub[1]) && (sim.μ_true <= lbub[2])
		end
		for r in sim.ests
	])
end

estimators = [
	sample_mean_variance_estimator,
	hill_estimator,
	gpd_estimator
]
estimator_names = [
	"Sample Mean / Var",
	"Hill",
	"GPD"
]
n_samples = [1000, 10000]
n_iterations = 5000
gpd_sims = Array{Simulation}(undef, length(n_samples), length(estimators))
pd_sims = Array{Simulation}(undef, length(n_samples), length(estimators))

for i in 1:length(n_samples)
	for j in 1:length(estimators)
		gpd_sims[i,j] = Simulation(
			gpd, 
			n_samples[i], 
			n_iterations,
			estimators[j]
		)
	end
end

for i in 1:length(n_samples)
	for j in 1:length(estimators)
		pd_sims[i,j] = Simulation(
			pd, 
			n_samples[i], 
			n_iterations,
			estimators[j]
		)
	end
end

#### Testing on Pareto data

First let's look at how the estimators perform on data that came from a Pareto distribution. As expected, @fig-evt-estimator-comparison-pareto shows that the baseline method of using the sample mean doesn't work well. It has skewed estimates, with high variance. However, both the hill estimator and the GPD estimator perform much better, with the estimates centred on the true population mean and with much lower variance.


In [ ]:
#| label: fig-evt-estimator-comparison-pareto
#| fig-cap: 'Distribution of estimates for a Pareto population distribution, with alpha=1.5 and theta=20.'

function plot_means(sim::Simulation)
	x = μ(sim.ests)
	p = histogram(
		x, 
		normalize=true, 
		label=L"\hat{\mu}",
		titlefontsize=10
	)
	vline!([sim.μ_true], label=L"\mu_\text{true}")
end

p = Array{Plots.Plot{Plots.GRBackend}}(undef, size(pd_sims))
for i in 1:length(n_samples)
	for j in 1:length(estimators)
		p[i,j] = plot_means(pd_sims[i, j])
		plot!(
			legend=(i==1) & (j==1), 
			title="$(estimator_names[j]) Estimator\n(X~Pareto, n=$(n_samples[i]))"
		)
	end
end

plot(p..., layout=(length(estimators), length(n_samples)))

Now we can quantify the total error that the estimator makes as the mean squared error:

$$
\text{MSE} = E[(\hat{\mu} - \mu)^2]
$$

This is a convenient metric to use because we can break it down in terms of bias and variance:

$$
\text{bias} = (E[\hat{\mu}] - E[\mu])^2
$$

$$
\text{variance} = E[(\hat{\mu} - E[\hat{\mu}])^2]
$$

$$
\text{MSE} = \text{bias} + \text{variance}
$$

Below you can see how the bias, variance and MSE compare. In this case the bias is low for all methods, while the Hill estimator has the lowest variance and MSE, closely followed by the GPD estimator. 


In [ ]:
#| label: fig-evt-estimator-stats-comparison-pareto
#| fig-cap: 'Average performance of estimators for a Pareto population distribution, with alpha=1.5 and theta=20. MSE is the Mean Squared Error of the estimates vs the true population mean, the variance is the variance of the estimates, and the bias is the squared difference between the average of the estimates and the true population mean. Ideally we want bias=0 and then MSE and variance as small as possible.'

function plot_stats!(sim::Simulation, label::Base.String)
	y = [bias(sim), variance(sim), mse(sim)]
	x = ["Bias", "Variance", "MSE"]
	p = plot!(
		x, 
		y,
		label=label,
		marker=true,
		line=false
	)
end

p = Array{Plots.Plot{Plots.GRBackend}}(undef, length(n_samples))
for i in 1:length(n_samples)
	p[i] = plot(
		legend = (i==1),
		titlefontsize=10, 
		title="X~Pareto, n=$(n_samples[i])"
	)
	for j in 1:length(estimators)
		plot_stats!(pd_sims[i, j], estimator_names[j])
	end
	hline!([0.0], color=:black, label="")
end

plot(p..., layout=grid(2,1))

Finally, just having a good estimate of the mean is not enough - we need to be able to estimate the standard error on the mean reliably for the estimator to be useful for A/B testing.

To validate whether the estimate of the standard error is good, I look at the distribution of standardised residuals, which is the difference between the estimated mean and the true population mean, divided by the standard error:

$$
\text{standardised residual} = \frac{\hat{\mu} - \mu}{\hat{\sigma}_\mu}
$$

If the standard errors are good, the standardised residuals should roughly follow a t-distribution with mean = 0, standard deviation = 1 and degrees of freedom = n - 1. 

We see below that the Hill estimator is good for both sample sizes, while the GPD works well for the larger sample size, but is not as good for the small sample size. The sample mean and variance is poor, again showing the skewed shape. 


In [ ]:
#| label: fig-evt-estimator-normed-comparison-pareto
#| fig-cap: 'A comparison of the standardised residuals of the different estimators for a Pareto population distribution. The t-test assumes that this statistic is approximately t-distributed, with d.o.f = samples size - 1. This expected distribution is shown in red. If the distribution matches, then you can expect the confidence intervals you calculate to have good coverage.'

function plot_standardized_errors(sim::Simulation)
	err = standardized_error(sim)
	p = histogram(
		err, 
		normalize=true, 
		label=L"(\hat{\mu} - \mu) / \hat{\sigma}",
		titlefontsize=10
	)
	plot!(TDist(sim.n_samples-1), label="Expected Distribution")
end

p = Array{Plots.Plot{Plots.GRBackend}}(undef, size(pd_sims))
for i in 1:length(n_samples)
	for j in 1:length(estimators)
		p[i,j] = plot_standardized_errors(pd_sims[i, j])
		plot!(
			legend=(i==1) & (j==1), 
			title="$(estimator_names[j]) Estimator\n(X~Pareto, n=$(n_samples[i]))"
		)
	end
end

plot(p..., layout=(length(estimators), length(n_samples)))

So overall the Hill and GPD estimators work well for Pareto distributed data, with the Hill estimator performing slightly better. But this is expected, because the data was drawn from the Pareto distribution, which is exactly what the Hill estimator expects. So what happens if the data comes from a GPD distribution?

#### Testing on GPD data

Here I do the same analysis again, but this time using data coming from a GPD distribution for my testing. This distribution is more complex, so will be a harder test for the Hill estimator, as it breaks the parametric assumption we made. 

First looking at the distributions of the mean estimates, we see again that the mean / variance estimator is very skewed, and the GPD estimator looks good. However, this time the Hill estimator is **very** bad. It's not skewed - the estimates have a nice, normally distributed shape, but they are not even nearly centred on the true population mean. 


In [ ]:
#| label: fig-evt-estimator-comparison-gpd
#| fig-cap: 'Distribution of estimates for a GPD population distribution, with mu=100, theta=20.0, xi=2/3.'

p = Array{Plots.Plot{Plots.GRBackend}}(undef, size(gpd_sims))
for i in 1:length(n_samples)
	for j in 1:length(estimators)
		p[i,j] = plot_means(gpd_sims[i, j])
		plot!(
			legend=(i==1) & (j==1), 
			title="$(estimator_names[j]) Estimator\n(X~GPD, n=$(n_samples[i]))"
		)
	end
end

plot(p..., layout=(length(estimators), length(n_samples)))

This is an example of the bias that can easily be introduced with this method. Because we made an assumption about the parametric form of the population distribution, and got that assumption wrong, the mean estimate is highly biased. 

You can see it clearly in the MSE and bias results in figure @fig-evt-estimator-stats-comparison-gpd. The hill estimator has low variance, but high bias, meaning that overall the MSE is much worse than the GPD estimator. Interestingly, even with this high bias it still has a lower MSE than the baseline sample mean / variance estimator, because the variance of the baseline mean/variance estimator is so high! This wouldn't hold in general though, probably with a higher sample size or slightly different parameters the hill estimator would perform the worst of all.


In [ ]:
#| label: fig-evt-estimator-stats-comparison-gpd
#| fig-cap: 'Average performance of estimators for a GPD population distribution, with mu=100, theta=20.0, xi=2/3. MSE is the Mean Squared Error of the estimates vs the true population mean, the variance is the variance of the estimates, and the bias is the squared difference between the average of the estimates and the true population mean. Ideally we want bias=0 and then MSE and variance as small as possible.'

p = Array{Plots.Plot{Plots.GRBackend}}(undef, length(n_samples))
for i in 1:length(n_samples)
	p[i] = plot(
		legend = (i==1),
		titlefontsize=10, 
		title="X~GPD, n=$(n_samples[i])"
	)
	for j in 1:length(estimators)
		plot_stats!(gpd_sims[i, j], estimator_names[j])
	end
	hline!([0.0], color=:black, label="")
end

plot(p..., layout=grid(2,1))

Finally we can check the standardised residuals again, and as expected the hill estimator is far away from the expected t-distribution. THe GPD estimator again works best, closely matching the t-distribution for the larger sample size. 


In [ ]:
#| label: fig-evt-estimator-normed-comparison-gpd
#| fig-cap: 'A comparison of the standardised residuals of the different estimators for a GPD population distribution. The t-test assumes that this statistic is approximately t-distributied, with d.o.f = samples size - 1. This expected distribution is shown in red. If the distribution matches, then you can expect the confidence intervals you calculate to have good coverage.'
p = Array{Plots.Plot{Plots.GRBackend}}(undef, size(gpd_sims))
for i in 1:length(n_samples)
	for j in 1:length(estimators)
		p[i,j] = plot_standardized_errors(gpd_sims[i, j])
		plot!(
			legend=(i==1) & (j==1), 
			title="$(estimator_names[j]) Estimator\n(X~GPD, n=$(n_samples[i]))"
		)
	end
end

plot(p..., layout=(length(estimators), length(n_samples)))

So it seems that the GPD distribution works better in general, but if the assumptions of the hill estimator are met (i.e. the samples come from a Pareto distribution), then the hill estimator is better. Of course, the GPD estimator will fail in the same way that the hill estimator did if we were to test it on a dataset that breaks its assumptions, it's just a more flexible distribution so its assumptions are less likely to break. 

## Appling extreme value theory to the trimmed mean

Now as both the Hill estimator and GDP estimator outperform the simple mean/variance estimator for power-law type data, we can try to apply them to our revenue A/B testing problem. However, we want to use the new estimators **only in the tail**. The rest of the distribution is unlikely to be power-law distributed, so the new estimators would fail there.

This is where the comparison with the trimmed and winsorized means comes in. In both of those methods, an arbitrary threshold is chosen, and all values that are above that threshold are designated as outliers and either ignored or capped. But what if we were to keep the outliers as they are, and use our new estimators on those, while the non-outliers are estimated by the regular mean-variance estimator. We can then combine the two estimates to get the overall estimate of the mean and the standard error for the whole sample.

::: {.callout-tip collapse="true"}
## Detailed method with maths

1. Set the trimming quartile to split the data into "outliers" and "inliers". This is an arbitrary percentile of the distribution, and typically only a small percentage of the data should be classified as outliers. 
2. Estimate the mean and standard error for the inliers and outliers separately. Now you should have the following stats:

$$
\hat{\mu}_\text{out}, \hat{\sigma}_{\mu, \text{out}} = \text{Estimated mean and standard error for outliers.}
$$

$$
\hat{\mu}_\text{in}, \hat{\sigma}_{\mu, \text{in}} = \text{Estimated mean and standard error for inliers.}
$$

$$
w_\text{in}, w_\text{out} = \text{The fraction of datapoints assigned to inliers / outliers.}
$$

$$
n = \text{Total number of datapoints in the sample.}
$$

3. Now we can combine the estimates to get the estimated mean and standard error for the whole sample. This method of combining the results together is built on top of the fact that the overall distribution can be thought of as a mixture of two distributions - the outliers and inliers. So we can combine the moments of the distribution as if they are a mixture (see [wikipedia](https://en.wikipedia.org/wiki/Mixture_distribution#Moments)).

$$
\hat{\mu} = w_\text{in} \hat{\mu}_\text{in} + w_\text{out} \hat{\mu}_\text{out}
$$

$$
\hat{\sigma}_\mu = \sqrt{w_\text{in}^2 \hat{\sigma}_{\mu, \text{in}}^2 + w_\text{out}^2 \hat{\sigma}_{\mu, \text{out}}^2 + \frac{w_\text{in} \hat{\mu}_\text{in}^2 + w_\text{in} \hat{\mu}_\text{in}^2 - \hat{\mu}}{n}}
$$

Note - the above equation for $\hat{\sigma}_\mu$ is slightly different to what you'll see in wikipedia. This is because we are combining the standard error on the mean, not the standard deviations.
:::

To test this out, I'll create a synthetic dataset which is a mixture of a GPD and two lognormal distributions. This will make the main distribution bimodal (and so not too easy for our method) but with a power law tail that gives it infinite variance. This distribution is shown in @fig-samples. 


In [ ]:
#| label: fig-samples
#| fig-cap: 'Histogram of samples from simulated data (blue), and the theoretical PDF (orange).'

data_dist = MixtureModel(
	[
	   LogNormal(2.0, 0.5),
	   LogNormal(3.8, 0.5),
	   GeneralizedPareto(0.0, 40.0, 0.7)
	], 
	[0.5, 0.4, 0.1]
);

x_samples = rand(data_dist, 1000);

x = LinRange(0.001,200,1000)
y_pdf = pdf.(data_dist, x)

p1 = histogram(
     x_samples, 
     normalize=true, 
     label="Samples"
)
plot!(x,y_pdf,label="PDF", lw=2)
xlabel!("X")

x = 10.0.^LinRange(-1,3,1000)
y_pdf = pdf.(data_dist, x)
y_comp = [pdf.(component(data_dist,i), x) .* probs(data_dist)[i] for i in 1:ncomponents(data_dist)]

p2 = plot(x,y_pdf,label="Mixture PDF", lw=2)
plot!(x, y_comp, lw=1, label=["Log-Norm 1" "Log-Norm 2" "GPD"])
plot!(yscale=:log10, xscale=:log10, yrange=[1e-6, 1.0])
xlabel!("X")

plot(p1,p2, size=(600,400))

Again I perform very similar simulations to the previous section - estimating the MSE, variance, bias and the MSE of the standardised residuals. In an ideal world, the MSE and variance will be as low as possible, the bias will be 0, and the MSE of the standardised residuals will be as close to 1 as possible. I also calculate the coverage of 95% confidence intervals of the estimates, which is the percentage of times that the true population mean is within the 95% confdience intervals of the estimate. If the estimator is working well it should cover the true value 95% of the time. I calculate all these metrics as a function of the trimming quantile.

Here I will compare the following estimators:

1. **The simple mean / variance estimator.** This is the standard method of estimating the population mean used in the t-test. It is independent of the trimming quantile.
1. **The trimmed mean / variance estimator.** The data is trimmed at the trimming quantile before being fed into the sample mean / variance estimator, meaning any samples above the trimming quantile are excluded.
1. **The winsorized mean / variance estimator.** The data is winsorized at the trimming quantile before being fed into the sample mean / variance estimator, meaning any samples above the trimming quantile are capped at the trimming quantile.
1. **The hill tail estimator.** This estimator uses the hill estimator for the samples above the trimming quantile, and the sample mean / variance estimator for the samples below, and then merges the results together.
1. **The GPD tail estimator.** This estimator uses the GPD estimator for the samples above the trimming quantile, and the sample mean / variance estimator for the samples below, and then merges the results together.

The simulation results are shown below in @fig-trimming-sim-by-metric.


In [ ]:
#| output: false

function tail_quantile_to_index(n_samples::Int64, q::Float64)
	trunc(Int, q * n_samples)
end

function merge_estimates(est1::EstimatorResult, est2::EstimatorResult, n1::Int64, n2::Int64)
	n = n1 + n2
	w1 = n1 / n
	w2 = n2 / n
	μ = est1.μ * w1 + est2.μ * w2

	σ = sqrt(
		w1^2*est1.σ^2 + w2*est2.σ^2 
		+ (w1 * est1.μ^2 + w2 * est2.μ^2 - μ^2) / n
	)
	return EstimatorResult(μ, σ)
end

function hill_tail_estimator(x_in::Vector{Float64}, tail_index::Int64)
	if tail_index <= 1
		return sample_mean_variance_estimator(x_in)
	end

	x_sorted = sort(x_in, rev=true)

	x_tail = x_sorted[1:tail_index]
	x_non_tail = x_sorted[tail_index+1:length(x_in)]

	tail_est = hill_estimator(x_tail)
	non_tail_est = sample_mean_variance_estimator(x_non_tail)

	n_tail = tail_index
	n = length(x_in)
	n_non_tail = n - n_tail

	return merge_estimates(
		tail_est, non_tail_est, n_tail, n_non_tail
	)
end

function gpd_tail_estimator(x_in::Vector{Float64}, tail_index::Int64)
	if tail_index <= 1
		return sample_mean_variance_estimator(x_in)
	end

	x_sorted = sort(x_in, rev=true)

	x_tail = x_sorted[1:tail_index]
	x_non_tail = x_sorted[tail_index+1:length(x_in)]

	tail_est = gpd_estimator(x_tail)
	non_tail_est = sample_mean_variance_estimator(x_non_tail)

	n_tail = tail_index
	n = length(x_in)
	n_non_tail = n - n_tail

	return merge_estimates(
		tail_est, non_tail_est, n_tail, n_non_tail
	)
end

function winsorized_estimator(x_in::Vector{Float64}, tail_index::Int64)
	if tail_index <= 1
		return sample_mean_variance_estimator(x_in)
	end

	x = sort(x_in, rev=true)

	x[1:tail_index] .= x[tail_index]
	return sample_mean_variance_estimator(x)
end

function trimmed_estimator(x_in::Vector{Float64}, tail_index::Int64)
	if tail_index <= 1
		return sample_mean_variance_estimator(x_in)
	end

	x = sort(x_in, rev=true)

	return sample_mean_variance_estimator(x[tail_index+1:length(x)])
end

struct TrimmingSimulation
	tail_quantiles::Vector{Float64}
	mse::Vector{Float64}
	variance::Vector{Float64}
	bias::Vector{Float64}
	mse_normed::Vector{Float64}
	variance_normed::Vector{Float64}
	coverage_95::Vector{Float64}
	function TrimmingSimulation(
		pop_dist::UnivariateDistribution,
		n_samples::Int64,
		n_iterations::Int64,
		estimator,
		tail_quantiles::Vector{Float64}
	)
		mse_result = Vector{Float64}(undef, length(tail_quantiles))
		variance_result = Vector{Float64}(undef, length(tail_quantiles))
		bias_result = Vector{Float64}(undef, length(tail_quantiles))
		mse_normed_result = Vector{Float64}(undef, length(tail_quantiles))
		variance_normed_result = Vector{Float64}(undef, length(tail_quantiles))
		coverage_95_result = Vector{Float64}(undef, length(tail_quantiles))
		for (i, q) in enumerate(tail_quantiles)
			tail_index = tail_quantile_to_index(
				n_samples, q
			)
			sim = Simulation(
				pop_dist, 
				n_samples,
				n_iterations, 
				estimator, 
				(tail_index,)
			)
			mse_result[i] = mse(sim)
			variance_result[i] = variance(sim)
			bias_result[i] = bias(sim)
			mse_normed_result[i] = mean(standardized_error(sim).^2)
			variance_normed_result[i] = std(standardized_error(sim))^2
			coverage_95_result[i] = coverage(sim, 0.05)
		end
		new(
			tail_quantiles,
			mse_result,
			variance_result,
			bias_result,
			mse_normed_result,
			variance_normed_result,
			coverage_95_result,
		)
	end
end

Random.seed!(123456); # I reset the seed before each simulation to ensure I am using the same samples each time.
baseline_trimming_sim = TrimmingSimulation(
	data_dist,
	50000,
	500,
	winsorized_estimator, # With trimming = 0 this is the same as untrimmed sample mean and variance 
	[0.0]
)

estimators = [trimmed_estimator, winsorized_estimator, hill_tail_estimator, gpd_tail_estimator]
estimator_names = [
	"Trimmed Mean"
	"Winsorized Mean"
	"Hill Tail Mean"
	"GPD Tail Mean"
]

trimming_sims = Vector{TrimmingSimulation}(undef, length(estimators))
for (i, estimator) in enumerate(estimators)
	println(i)
	Random.seed!(123456); # I reset the seed before each simulation to ensure I am using the same samples each time.
	trimming_sims[i] = TrimmingSimulation(
		data_dist,
		50000,
		500,
		estimator,
		10.0.^LinRange(-3,log10(0.4),20)
	)
end

In [ ]:
#| label: fig-trimming-sim-by-metric
#| fig-cap: 'Performance of the different trimming methods based on various metrics. The MSE and variance should be as small as possible. The Bias should be as close to 0.0 as possible, the Standardised MSE should be 1.0 and the 95% coverage should be 0.95.'

function plot_sim_metrics!(sims::Vector{TrimmingSimulation}, sim_bl::TrimmingSimulation, metric::Symbol)
	hline!(getfield(sim_bl, metric))
	for (i, sim) in enumerate(sims)
		plot!(
			sim.tail_quantiles, 
			getfield(sim, metric),
			marker=true,
			markersize=2
		)
	end
end
estimator_names = ["Trimmed Mean" "Winsorized Mean" "Hill Tail Mean" "GPD Tail Mean"]

function percent_fmt(x)
	if x < 0.01
		"$(round(x*100, digits=1))%"
	else
		"$(trunc(Int, x*100))%"
	end
end

xticklocations = [0.001, 0.01, 0.1, 0.5]

p1 = plot(
	title="MSE",
	legend=false,
	xlabel="Trimming Quantile",
	yscale=:log10,
	xscale=:log10,
	titlefontsize=10, 
	xlabelfontsize=8,
	xformatter = percent_fmt, 
	xticks=xticklocations,
	xrange = (0.001, 0.5)
)
plot_sim_metrics!(trimming_sims, baseline_trimming_sim, :mse)

p2 = plot(
	title="Variance",
	legend=false,
	xlabel="Trimming Quantile",
	yscale=:log10,
	xscale=:log10,
	titlefontsize=10, 
	xlabelfontsize=8,
	xformatter = percent_fmt, 
	xticks=xticklocations,
	xrange = (0.001, 0.5)
)
plot_sim_metrics!(trimming_sims, baseline_trimming_sim, :variance)

p3 = plot(
	title="Bias",
	legend=false,
	xlabel="Trimming Quantile",
	titlefontsize=10, 
	xlabelfontsize=8,
	yscale=:log10,
	xscale=:log10,
	xformatter = percent_fmt, 
	xticks=xticklocations,
	xrange = (0.001, 0.5)
)
plot_sim_metrics!(trimming_sims, baseline_trimming_sim, :bias)

p4 = plot(
	title="Standardised MSE",
	legend=false,
	xlabel="Trimming Quantile",
	yscale=:log10,
	titlefontsize=10, 
	xlabelfontsize=8,
	xscale=:log10,
	xformatter = percent_fmt, 
	xticks=xticklocations,
	xrange = (0.001, 0.5)
)
plot_sim_metrics!(trimming_sims, baseline_trimming_sim, :mse_normed)
hline!([1.0], color=:black, label="Ideal")

p5 = plot(
	title="95% CI Coverage",
	legend=false,
	xlabel="Trimming Quantile",
	titlefontsize=10, 
	xlabelfontsize=8,
	xscale=:log10,
	xformatter = percent_fmt, 
	yformatter = percent_fmt,
	xticks=xticklocations,
	xrange = (0.001, 0.5)
)
plot_sim_metrics!(trimming_sims, baseline_trimming_sim, :coverage_95)
hline!([0.95], color=:black, label="Ideal")

p6 = plot(
	showaxis=false,
	grid=false,
	legend = :inside, 
	framestyle = :none,
	legendtitle = "Estimator",
	legendfontpointsize = 10
)
plot!(
	[0],
	label="Simple Mean"
)
plot!(
	[0 0 0 0],
	marker=true,
	label=estimator_names
)
xlims!(-0.5, 0.5)
ylims!(-0.5, 0.5)

plot(p1,p2,p3,p4,p5,p6)

These results are interesting and in my opinion very promising for this approach!

* The simple mean is the least biased of all the methods, but does not have good coverage or standardised MSE, indicating that the confidence intervals would not work well for this estimator, breaking our A/B tests.
* The trimmed and winsorized means both behave as expected. The more we trim, the bias increases, and the variance decreases. Overall, the increase in bias is larger than the decrease in variance, meaning that the MSE gets worse. And the bias is so large that the coverage is terrible - the confidence intervals never contain the true population mean. This demonstrates why the trimmed and winsorized means are so difficult to use, because while the variance decreases, you introduce significant bias.
* Our new estimators seem to work well, as long as the trimming quantile is not too large. Both have good coverage and standardised MSE, meaning that can be more reliably used as A/B testing metrics. The variance, bias and MSE are closer to the values for the simple mean. This means we get similar performance on average, but with a better behaved estimator that we can construct reliable confidence intervals for.
* The hill tail estimator works better than the GPD tail estimator as long as the trimming quantile is kept below ~1%. Above that, the coverage of the hill estimator deteriorates, while the GPD tail estimator maintains its coverage until ~10%. This is expected because the GPD tail estimator is a more general distribution, so can describe a bigger portion of the tail which is not exactly Pareto distributed. 
* The GPD tail estimator is less robust. This is likely because of the numerical estimation method. Between 1% and 10% the variance, bias and MSE go extremely high. 

Overall, based on this, I think the hill estimator has the most promise. While it breaks down at high trimming quantiles, as long as the correct trimming quantile is chosen it works well. It is also computationally simpler than the GPD estimator, because it has an analytical solution.

## Summary

So it looks like we might be able to do better than the trimmed or winsorized means! This is still early stages, more work needs to be done to test this, but overall the hill-tail estimator has better coverage than the regular sample mean, while maintaining a good level of bias and variance.

Some interesting next steps would be to select the trimming quantile automatically, for example using a similar method to @powerlawsempirical, as the hill estimator is very sensitive to this choice, or exploring bias corrections to the MLE estimates. 

## Execution Details


In [ ]:
#| echo: false
#| output: true
println("Installed packages:")
for (uuid, dep) in Pkg.dependencies()
	dep.is_direct_dep || continue
	dep.version === nothing && continue
	println(dep.name," (", dep.version, ")")
end
println("")
print("Execution time: ")
println(canonicalize(now() - START_TIME))